In [ ]:
import pandas as pd
import random

# Define age groups
age_groups = ["15-25", "26-35", "36-45", "46-60"]

# Define locations and their corresponding climates with at least 3 cities per state and UT
locations = {
    "Mumbai": "Humid", "Pune": "Humid", "Nagpur": "Humid",
    "Delhi": "Extreme", "Noida": "Extreme", "Gurgaon": "Extreme",
    "Bangalore": "Moderate", "Mysore": "Moderate", "Hubli": "Moderate",
    "Chennai": "Hot & Humid", "Coimbatore": "Hot & Humid", "Madurai": "Hot & Humid",
    "Kolkata": "Tropical", "Siliguri": "Tropical", "Durgapur": "Tropical",
    "Shimla": "Cold", "Manali": "Cold", "Dharamshala": "Cold",
    "Goa": "Coastal", "Panaji": "Coastal", "Margao": "Coastal",
    "Jaipur": "Dry & Hot", "Jodhpur": "Dry & Hot", "Udaipur": "Dry & Hot",
    "Hyderabad": "Warm", "Warangal": "Warm", "Nizamabad": "Warm",
    "Aizawl": "Tropical", "Agartala": "Tropical", "Imphal": "Tropical",
    "Port Blair": "Coastal", "Daman": "Coastal", "Kavaratti": "Coastal",
    "Leh": "Cold", "Kargil": "Cold", "Dras": "Cold",
    "Guwahati": "Humid", "Dibrugarh": "Humid", "Silchar": "Humid",
    "Patna": "Extreme", "Gaya": "Extreme", "Muzaffarpur": "Extreme",
    "Bhopal": "Moderate", "Indore": "Moderate", "Jabalpur": "Moderate",
    "Lucknow": "Extreme", "Kanpur": "Extreme", "Varanasi": "Extreme",
    "Chandigarh": "Moderate", "Mohali": "Moderate", "Panchkula": "Moderate",
    "Ranchi": "Tropical", "Jamshedpur": "Tropical", "Dhanbad": "Tropical",
    "Itanagar": "Tropical", "Shillong": "Tropical", "Kohima": "Tropical",
    "Dehradun": "Cold", "Nainital": "Cold", "Mussoorie": "Cold",
    "Vijayawada": "Warm", "Vizag": "Coastal", "Chittoor": "Moderate"
}

# Define recommendations based on climate
recommendations = {
    "Humid": {"travel": "Matheran", "food": "Kokum Sharbat", "wellness": "Cooling Yoga"},
    "Extreme": {"travel": "Nainital", "food": "Lassi", "wellness": "Meditation"},
    "Moderate": {"travel": "Coorg", "food": "Dosa", "wellness": "Morning Walks"},
    "Hot & Humid": {"travel": "Ooty", "food": "Buttermilk", "wellness": "Hydrotherapy"},
    "Tropical": {"travel": "Sundarbans", "food": "Fish Curry", "wellness": "Ayurvedic Massage"},
    "Cold": {"travel": "Gulmarg", "food": "Kahwa", "wellness": "Hot Stone Therapy"},
    "Coastal": {"travel": "Andaman", "food": "Seafood Platter", "wellness": "Beach Yoga"},
    "Dry & Hot": {"travel": "Mount Abu", "food": "Aam Panna", "wellness": "Cooling Spa"},
    "Warm": {"travel": "Araku Valley", "food": "Ragi Mudde", "wellness": "Evening Meditation"}
}

# Generate dataset with different age groups for each city
users_data = []
for location, climate in locations.items():
    for age_group in age_groups:
        for preference in ["travel", "food", "wellness"]:
            recommendation = recommendations[climate][preference]
            users_data.append([age_group, location, climate, preference, recommendation])

# Create DataFrame
df = pd.DataFrame(users_data, columns=["Age Group", "Location", "Climate", "Preference", "Recommendation"])

# Save to CSV
df.to_csv("india_recommendation_dataset.csv", index=False)
print("Dataset generated successfully!")


Dataset generated successfully!


In [8]:
!curl ifconfig.me

34.73.23.25

In [5]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [6]:
import pandas as pd
import random
import numpy as np
import streamlit as st
from textblob import TextBlob
from difflib import get_close_matches
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load dataset
df = pd.read_csv("/content/india_recommendation_dataset (3).csv")

# Train ML model
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["Location"] + " " + df["Climate"] + " " + df["Age Group"])
y = df["Recommendation"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Chatbot function
def chatbot(location, age_group, sentiment_text):
    if location not in df["Location"].values:
        closest_match = get_close_matches(location, df["City"].unique(), n=1)
        if closest_match:
            location = closest_match[0]
        else:
            return "Sorry, we don't have data for your location."

    climate = df[df["Location"] == location]["Climate"].values[0]
    sentiment = TextBlob(sentiment_text).sentiment.polarity
    preference = "wellness" if sentiment < -0.2 else "food" if sentiment < 0.2 else "travel"

    query_vector = vectorizer.transform([f"{location} {climate} {age_group}"])
    prediction = clf.predict(query_vector)[0]

    return f"For age group {age_group} in {location} ({climate} climate), we recommend: {prediction} ({preference})."

# Streamlit UI
st.title("AI Recommendation Chatbot 🤖")
st.write("Enter your details below to get personalized recommendations.")

location = st.text_input("Enter City:")
age_group = st.selectbox("Select Age Group:", df["Age Group"].unique())
sentiment_text = st.text_area("How do you feel today?")

if st.button("Get Recommendation"):
    if location and sentiment_text:
        result = chatbot(location, age_group, sentiment_text)
        st.success(result)
    else:
        st.error("Please enter all details.")

# Run this script in Colab:
# !streamlit run app.py & npx localtunnel --port 8501


2025-03-23 13:05:20.280 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 13:05:20.487 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-23 13:05:20.488 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 13:05:20.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 13:05:20.492 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 13:05:20.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 13:05:20.495 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-23 13:05:20.496 Thread 'MainThread': mi

In [ ]:
!streamlit run /content/app_1.py & npx localtunnel --port 8501